In [0]:
# creamos lista de strings
palabras = sc.parallelize(['HOLA', 'Que', 'TAL', 'Bien'])
# creamos con map una lista que dé la palabra, y la longitud del string de la palabra
pal_long = palabras.map(lambda elem: (elem, len(elem)))
# mostramos
pal_long.collect()

Out[13]: [('HOLA', 4), ('Que', 3), ('TAL', 3), ('Bien', 4)]

In [0]:
frutas = sc.parallelize([('P', 'peras,piñas'), ('M', 'manzanas,melones,melocotones'), ('N','naranjas,nectarinas'), ('L', 'limones')])

In [0]:
# creamos una lista de frutas y su letra por la que empiezan
frutas = sc.parallelize([('P', 'peras,piñas'), ('M', 'manzanas,melones,melocotones'), ('N','naranjas,nectarinas'), ('L', 'limones')])
# transformamos los strings de las frutas en mayusculas
frutas_MAY = frutas.mapValues(lambda elemento: elemento.upper())
# mostramos
frutas_MAY.collect()

Out[15]: [('P', 'PERAS,PIÑAS'),
 ('M', 'MANZANAS,MELONES,MELOCOTONES'),
 ('N', 'NARANJAS,NECTARINAS'),
 ('L', 'LIMONES')]

In [0]:
# podemos utilizar también reduceByKey para reducir por claves, pero se tiene que tener en cuenta la función de reducción. creamos rdd
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4)])
# reducimos por claves, sumando los valores de las claves
rr = r.reduceByKey(lambda v1,v2: v1+v2)
# mostramos la reducción
print(rr.collect())


[('B', 5), ('C', 4), ('A', 2)]


#####CONSECUENCIAS
#####No seguir las normas de la función de reducción (REDUCE / REDUCEBYKEY)

In [0]:
# aquí podemos comprobar las consecuencias de no entender bien la función de reducción. aparentemente tenemos un rdd similar al anterior, que tendría que dar parecido
r = sc.parallelize([('A', 1),('C', 4),('A', 1),('B', 1),('B', 4), ('B', 7)])
# reducimos por clave
rr = r.reduceByKey(lambda v1,v2: str(v1+v2))
# mostramos
print(rr.collect())

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3673782311700150>:6
      4 rr = r.reduceByKey(lambda v1,v2: str(v1+v2))
      5 # mostramos
----> 6 print(rr.collect())

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/rdd.py:1825, in RDD.collect(self)
   1823 with SCCallSiteSync(self.context):
   1824     assert self.ctx._jvm is not None
-> 1825     sock_info = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
   1826 return list(_load_from_socket(sock_info, self._jrdd_deserializer))

File /databricks/spar

In [0]:
# observamos un error porque no puede devolver una lista ya que no es de tipo compatible

In [0]:
# para ello mejor usar .groupByKey() en estos casos. creamos rdd
r = sc.parallelize([('A', 1),('C', 2),('A', 3),('B', 4),('B', 5)])
# mostramos los valores de la lista agrupados por sus claves, y también las claves y la longitud de los valores de cada clave
print(r.groupByKey().mapValues(list).collect())
print(r.groupByKey().mapValues(len).collect())

[('B', [4, 5]), ('C', [2]), ('A', [1, 3])]
[('B', 2), ('C', 1), ('A', 2)]


In [0]:
# creamos una rdd como antes, de claves-valor
rdd = sc.parallelize([('A',1),('B',6),('C',3),('A',4),('A',5),('B',2)])
# ahora queremos ordenarlas por las claves, donde el False es de forma inversa
res = rdd.sortByKey(False)
# mostramos
res.collect()

Out[6]: [('C', 3), ('B', 6), ('B', 2), ('A', 1), ('A', 4), ('A', 5)]

In [0]:
# para las transformaciones join, necesitamos tener listas con sentido. creamos dos rdds que tengan valores clave en común. aunque en las join() simplemente se unirán sin relación, poco útil para analizar en un futuro con otras transformaciones.
rdd1 = sc.parallelize([('A',1),('B',2)])
rdd2 = sc.parallelize([('A',4),('B',5),('A',7)])
# unimos rdd1 como la izaquierda con un join() de rdd2 de la derecha
rddjoin = rdd1.join(rdd2)
# mostramos
rddjoin.collect()


Out[9]: [('B', (2, 5)), ('A', (1, 4)), ('A', (1, 7))]

In [0]:
# observamos ahora diferentes tipos de join(). generamos más listas
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('A',5),('B',6),('D',7)])
# hacemos una inner join() con rdd1 de izquierda y rdd2 de derecha
rdd_join = rdd1.join(rdd2)
# mostramos
rdd_join.collect()
# vemos que es igual que lo anterior, donde no sale la parte de la 'D',7 porque no es la parte de la izquierda. lo vemos cambiándolos de lado: 

Out[10]: [('B', (2, 6)), ('A', (1, 4)), ('A', (1, 5))]

In [0]:
# mismas listas
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('A',5),('B',6),('D',7)])
# hacemos una left outer join con rdd1 de izquierda y rdd2 de derecha
rdd_ljoin = rdd1.leftOuterJoin(rdd2)
# mostramos
rdd_ljoin.collect()

Out[12]: [('B', (2, 6)), ('C', (3, None)), ('A', (1, 4)), ('A', (1, 5))]

In [0]:
# vemos que se muestran las claves de rdd1 con los valores que se tienen de rdd2 uniéndolos

In [0]:
# mismas listas
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('A',5),('B',6),('D',7)])
# ahora hacemos una right outer join, con rdd1 de izquierda y rdd2 de derecha, pero ahora las cogemos al revés
rdd_rjoin = rdd1.rightOuterJoin(rdd2)
# mostramos
rdd_rjoin.collect()

Out[13]: [('B', (2, 6)), ('D', (None, 7)), ('A', (1, 4)), ('A', (1, 5))]

In [0]:
# se observa como se han cogido las claves de rdd2 y se han unido con los valores de las claves de rdd1 que coinciden

In [0]:
# mismas listas
rdd1 = sc.parallelize([('A',1),('B',2),('C',3)])
rdd2 = sc.parallelize([('A',4),('A',5),('B',6),('D',7)])
# ahora hacemos una full outer join para observar más diferencias entre las join()
rdd_rjoin = rdd1.fullOuterJoin(rdd2)
# mostramos
rdd_rjoin.collect()

Out[14]: [('B', (2, 6)),
 ('D', (None, 7)),
 ('C', (3, None)),
 ('A', (1, 4)),
 ('A', (1, 5))]

In [0]:
# observamos que se unen todo, todas las claves y todos los valores de las dos listas

###EJERCICIO PARES CLAVE VALOR
####Procesar datos ventas ejercicio actual y compararlos con los del ejercicio anterior

In [0]:
# primer escenario
# ventas ejercicio actual, mes de Abril, todas las tiendas
rdd1 = sc.parallelize([('Nike',51805),('Puma',42329),('Adidas',63542), ('Puma',27923),('Nike',75335),('Puma',45102),('Adidas',49583), ('Puma',37869),('Adidas',54201), ('Puma',31582),('Nike',62747)])
# ventas ejercicio anterior (historico), del 2DO TRIMESTRE
rdd2 = sc.parallelize([('Nike',224589),('Adidas',219123),('Puma',166524)])

In [0]:
# reducimos los valores del ejercicio actual del mes de abril en una sola clave, sumando los valores de cada clave igual
rdd1_resumen = rdd1.reduceByKey(lambda elem1, elem2: elem1 + elem2)
rdd1_resumen.collect()
# ahora tenemos todos los datos de abril recogidos y con relación con rdd2, ahora los unimos:

Out[35]: [('Nike', 189887), ('Puma', 184805), ('Adidas', 167326)]

In [0]:
# hacemos una comparación del ejercicio actual del mes de abril comparado con todo el segundo cuatrimestre, guardados por marcas, siendo la primera abril y la segunda las ventas del 2º trimestre
rdd_comparar = rdd1_resumen.join(rdd2)
rdd_comparar.collect()

Out[36]: [('Nike', (189887, 224589)),
 ('Puma', (184805, 166524)),
 ('Adidas', (167326, 219123))]

In [0]:
# ventas ejercicio actual, mes de Abril, todas las tiendas con además una nueva marca como clave
rdd1 = sc.parallelize([('Nike',51805),('Puma',42329),('Novedad',18536), ('Adidas',63542),('Puma',27923),('Nike',75335),('Puma',45102), ('Adidas',49583), ('Puma',37869),('Novedad',27196), ('Adidas',54201), ('Puma',31582),('Nike',62747),('Novedad',25409)])

# ventas ejercicio anterior (historico), del 2DO TRIMESTRE
rdd2 = sc.parallelize([('Nike',224589),('Adidas',219123),('Puma',166524)])

In [0]:
# recolectamos las marcas de nuevo como claves
rdd1_resumen2 = rdd1.reduceByKey(lambda elem1, elem2: elem1 + elem2)
# mostramos las marcas como claves y sus valores como la suma total 
rdd1_resumen2.collect()

Out[38]: [('Nike', 189887), ('Puma', 184805), ('Adidas', 167326), ('Novedad', 71141)]

In [0]:
# hacemos una comparación del mes de abril comparado con todo el segundo cuatrimestre, guardados por marcas, siendo la primera abril y la segunda las ventas del 2º trimestr
rdd_comparar2 = rdd1_resumen2.leftOuterJoin(rdd2)
rdd_comparar2.collect()

Out[41]: [('Nike', (189887, 224589)),
 ('Novedad', (71141, None)),
 ('Puma', (184805, 166524)),
 ('Adidas', (167326, 219123))]

In [0]:
# observamos que hemos unido la lista de rdd1 con las claves reducidas y los valores como la suma total, unida con rdd2 que son las claves y los valores totales de las ventas trimestrales. Vemos que como en rdd2 no teníamos 'Novedad', el segundo valor en la lista de esa clave es None porque no existe en rdd2